In [ ]:
import param
import panel as pn

pn.extension('katex')

The [Param user guide](Param.ipynb) described how to set up classes which declare parameters and link them to some computation or visualization. In this section we will discover how to connect multiple such panels into a ``Pipeline`` to express complex workflows where the output of one stage feeds into the next stage. To start using a ``Pipeline`` let us declare an empty one by instantiating the class:

In [ ]:
pipeline = pn.pipeline.Pipeline()

Having set up a Pipeline it is now possible to start populating it. While we have already seen how to declare a ``Parameterized`` class with parameters which are linked to some visualization or computation on a method using the ``param.depends`` decorator, ``Pipelines`` make use of another decorator and a convention for displaying the objects.

The ``param.output`` decorator provides a way to annotate the methods on a class by declaring its outputs. A ``Pipeline`` uses this information to determine what outputs are available to be fed into the next stage of the workflow. In the example below the ``Stage1`` class has two parameters of its own (``a`` and ``b``) and one output, which is named ``c``. The signature of the decorator allows a number of different ways of declaring the outputs:

* ``param.output()``: Declaring an output without arguments will declare that the method returns an output which will inherit the name of the method and does not make any specific type declarations.
* ``param.output(param.Number)``: Declaring an output with a specific ``Parameter`` or Python type also declares an output with the name of the method but declares that the output will be of a specific type.
* ``param.output(c=param.Number)``: Declaring an output using a keyword argument allows overriding the method name as the name of the output and declares the type.

It is also possible to declare multiple parameters, either as keywords (Python >= 3.6 required) or tuples:

* ``param.output(c=param.Number, d=param.String)`` or ``param.output(('c', param.Number), ('d', param.String))``

In the example below the output is simply the result of multiplying the two inputs (``a`` and ``b``) which will produce output ``c``. Additionally we declare a ``view`` method which returns a ``LaTeX`` pane which will render the equation to ``LaTeX``. Finally a ``panel`` method declares returns a Panel object rendering both the parameters and the view; this is the second convention that a ``Pipeline`` expects.

Let's start by displaying this stage on its own:

In [ ]:
class Stage1(param.Parameterized):
    
    a = param.Number(default=5, bounds=(0, 10))

    b = param.Number(default=5, bounds=(0, 10))
    
    @param.output(('c', param.Number), ('d', param.Number))
    def output(self):
        return self.a * self.b, self.a ** self.b
    
    @param.depends('a', 'b')
    def view(self):
        c, d = self.output()
        return pn.pane.LaTeX('${a} * {b} = {c}$\n${a}^{{{b}}} = {d}$'.format(
            a=self.a, b=self.b, c=c, d=d), style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)
    
stage1 = Stage1()
stage1.panel()

To summarize we have followed several conventions when setting up this stage of our ``Pipeline``:

1. Declare a Parameterized class with some input parameters
2. Declare one or more output methods and name them appropriately
3. Declare a ``panel`` method that returns a view of the object that the ``Pipeline`` can render

Now that the object has been instantiated we can also query it for its outputs:

In [ ]:
stage1.param.outputs()

We can see that ``Stage1`` declares an output of name ``c`` of ``Number`` type which can be accessed by calling the ``output`` method on the object. Now let us add this stage to our ``Pipeline`` using the ``add_stage`` method:

In [ ]:
pipeline.add_stage('Stage 1', stage1)

A ``Pipeline`` with only a single stage is not much of a ``Pipeline`` of course, so it's time to set up a second stage, consuming the outputs of the first. Recall that ``Stage1`` declares one output named ``c``. This means that if the output from ``Stage1`` should flow to ``Stage2``, the latter should declare a ``Parameter`` named ``c`` that will consume the output of the first stage. It does not have to consume all parameters, so we can ignore output ``d``.

Below we therefore define parameters ``c`` and ``exp`` and since ``c`` is the output of the first stage the ``c`` parameter will be declared with a negative precedence stopping Panel from generating a widget for it. In other respects this class is very similar to the first one; it declares both a ``view`` method that depends on the parameters of the class and a ``panel`` method that returns a view of the object.

In [ ]:
class Stage2(param.Parameterized):
    
    c = param.Number(default=5, precedence=-1, bounds=(0, None))

    exp = param.Number(default=0.1, bounds=(0, 3))
    
    @param.depends('c', 'exp')
    def view(self):
        return pn.pane.LaTeX('${%s}^{%s}={%.3f}$' % (self.c, self.exp, self.c**self.exp),
                             style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)
    
stage2 = Stage2(c=stage1.output()[0])
stage2.panel()

Now that we have declared the second stage of the pipeline, let us add it to the ``Pipeline`` object:

In [ ]:
pipeline.add_stage('Stage 2', stage2)

And that's it; we have no declared a two stage pipeline, where the output ``c`` flows from the first stage into the second stage. To display it we can now view the ``pipeline.layout``:

In [ ]:
pipeline.layout

As you can see the ``Pipeline`` renders a little diagram displaying the available stages in the workflow along with previous and next buttons to move between each stage. This allows setting up complex workflows with multiple stages, where each component is a self-contained unit, with minimal declarations about its outputs (using the ``param.output`` decorator) and how to render it (by declaring a ``panel`` method).

Above we created the ``Pipeline`` as we went along which makes some sense in a notebook to allow debugging and development of each stage.  When deploying the Pipeline as a server app or when there's no reason to instantiate each stage separately, it is also possible to declare the stages as part of the constructor:

In [ ]:
stages = [
    ('Stage 1', Stage1),
    ('Stage 2', Stage2)
]

pipeline = pn.pipeline.Pipeline(stages)
pipeline.layout

As you will note the Pipeline stages may be either ``Parameterized`` instances or ``Parameterized`` classes, but when working with instances you must ensure that updating the parameters of the class is sufficient to update the current state of the class.